In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf


In [2]:
import tensorflow.contrib.slim as slim


def generator(z):
    
    zP = slim.fully_connected(z,4*4*256,normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_project',weights_initializer=initializer)
    zCon = tf.reshape(zP,[-1,4,4,256])
    
    gen1 = slim.convolution2d_transpose(\
        zCon,num_outputs=64,kernel_size=[5,5],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv1', weights_initializer=initializer)
    
    gen2 = slim.convolution2d_transpose(\
        gen1,num_outputs=32,kernel_size=[5,5],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv2', weights_initializer=initializer)
    
    gen3 = slim.convolution2d_transpose(\
        gen2,num_outputs=16,kernel_size=[5,5],stride=[2,2],\
        padding="SAME",normalizer_fn=slim.batch_norm,\
        activation_fn=tf.nn.relu,scope='g_conv3', weights_initializer=initializer)
    
    g_out = slim.convolution2d_transpose(\
        gen3,num_outputs=1,kernel_size=[32,32],padding="SAME",\
        biases_initializer=None,activation_fn=tf.nn.tanh,\
        scope='g_out', weights_initializer=initializer)
    
    return g_out

In [3]:
def discriminator(bottom, reuse=False):
    
    dis1 = slim.convolution2d(bottom,16,[4,4],stride=[2,2],padding="SAME",\
        biases_initializer=None,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv1',weights_initializer=initializer)
    
    dis2 = slim.convolution2d(dis1,32,[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv2', weights_initializer=initializer)
    
    dis3 = slim.convolution2d(dis2,64,[4,4],stride=[2,2],padding="SAME",\
        normalizer_fn=slim.batch_norm,activation_fn=lrelu,\
        reuse=reuse,scope='d_conv3',weights_initializer=initializer)
    
    d_out = slim.fully_connected(slim.flatten(dis3),1,activation_fn=tf.nn.sigmoid,\
        reuse=reuse,scope='d_out', weights_initializer=initializer)
    
    return d_out

In [4]:
import scipy

#This function performns a leaky relu activation, which is needed for the discriminator network.
def lrelu(x, leak=0.2, name="lrelu"):
     with tf.variable_scope(name):
         f1 = 0.5 * (1 + leak)
         f2 = 0.5 * (1 - leak)
         return f1 * x + f2 * abs(x)

In [5]:


tf.reset_default_graph()

z_size = 100 #Size of z vector used for generator.

#This initializaer is used to initialize all the weights of the network.
initializer = tf.truncated_normal_initializer(stddev=0.02)

#These two placeholders are used for input into the generator and discriminator, respectively.
z_in = tf.placeholder(shape=[None,z_size],dtype=tf.float32) #Random vector
real_in = tf.placeholder(shape=[None,32,32,1],dtype=tf.float32) #Real images

Gz = generator(z_in) #Generates images from random z vectors
Dx = discriminator(real_in) #Produces probabilities for real images
Dg = discriminator(Gz,reuse=True) #Produces probabilities for generator images

#These functions together define the optimization objective of the GAN.
d_loss = -tf.reduce_mean(tf.log(Dx) + tf.log(1.-Dg)) #This optimizes the discriminator.
g_loss = -tf.reduce_mean(tf.log(Dg)) #This optimizes the generator.

tvars = tf.trainable_variables()

#The below code is responsible for applying gradient descent to update the GAN.
trainerD = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
trainerG = tf.train.AdamOptimizer(learning_rate=0.0002,beta1=0.5)
d_grads = trainerD.compute_gradients(d_loss,tvars[9:]) #Only update the weights for the discriminator network.
g_grads = trainerG.compute_gradients(g_loss,tvars[0:9]) #Only update the weights for the generator network.

update_D = trainerD.apply_gradients(d_grads)
update_G = trainerG.apply_gradients(g_grads)

In [6]:
def save_images(images, size, image_path):
    return imsave(inverse_transform(images), size, image_path)

def imsave(images, size, path):
    return scipy.misc.imsave(path, merge(images, size))

def inverse_transform(images):
    return (images+1.)/2.

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h * size[0], w * size[1]))

    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w] = image

    return img

In [7]:
# i = 0
# def saveBatch(images):
#     for image_ in images:
# #         print(np.squeeze(image_, axis=(2,)).shape)
#         global i
#         image = np.squeeze(image_, axis=(2,))
#         if not os.path.exists('/home/paragjain/Desktop/GAN_Trial/generator_output_image'):
#             os.makedirs('/home/paragjain/Desktop/GAN_Trial/generator_output_image')
#         cv2.imwrite('/home/paragjain/Desktop/GAN_Trial/generator_output_image/output'+str(i)+'.jpeg', image)
#         i = i+1
        

In [8]:
def generate_flattened_dataset():
    dataset = np.arange(784).reshape((1,784))
    # directory path where all the images for training are stored. These images are expected to be
    # i) Grayscale
    # ii) Each of size 28 x 28
    
    for imageName in os.listdir('/home/paragjain/Desktop/GAN_Trial/resize_code/ALL_RESIZED'):
        image = cv2.imread('/home/paragjain/Desktop/GAN_Trial/resize_code/ALL_RESIZED/'+imageName, 0)
        image_flattened = image.flatten()
#         print(image_flattened.shape)
        dataset = np.vstack((dataset, image_flattened))


    dataset = np.delete(dataset, 0, axis=0)
    print(dataset.shape)

    return dataset

dataset = generate_flattened_dataset()


(1197, 784)


In [9]:


# images = []
# for root, subdirs, files in os.walk('/home/paragjain/Desktop/GAN Trial/all_resized'):
#     if files:
#         # print(files)
#         images.extend([root+str('/')+image for image in files])

# # print(len(images))

# dataset = np.arange(784).reshape((1,784))
# for imagePath in images:
#     # print(imagePath)
#     image = cv2.imread(imagePath,0)
#     # imagePathArr = imagePath.split('/')
#     # destPath = "/".join(imagePathArr[0:len(imagePathArr)-1])+"_resized/"+imagePathArr[len(imagePathArr)-1]
#     # destPath2 = "/".join(imagePathArr[0:len(imagePathArr)-3])+"/all_resized/"+imagePathArr[len(imagePathArr)-1]
#     # destPath = '/home/paragjain/Desktop/'+imagePathArr[len(imagePathArr)-1]
#     # print(destPath2)
#     # print()
#     # image_resize = cv2.resize(image, (28,28), interpolation=cv2.INTER_CUBIC)
#     image_resize_flatten = image.flatten()
#     # print(image_resize_flatten.shape)
#     dataset = np.vstack((dataset, image_resize_flatten))
#     # cv2.imshow('image', image)
#     # cv2.imwrite(destPath, image_resize)
#     # cv2.imwrite(destPath2, image_resize)

# # print(len(dataset))
# dataset = np.delete(dataset, 0, axis=0)
# print(dataset.shape)




batch_size = 36 # Min : 36
iterations = 11
# z_size = 100
start = 0

init = tf.global_variables_initializer()
saver = tf.train.Saver()

sess = tf.Session()
sess.run(init)
count = 0
for i in range(iterations):
    count = count + 1
    zs = np.random.uniform(-1.0, 1.0, size=[batch_size, z_size]).astype(np.float32)
    xs = dataset[start:start+batch_size]
    start = start+batch_size
    # print(xs.shape, type(xs))
    xs_reshaped = np.reshape(xs, (batch_size, 28, 28, 1))
#     print(xs_reshaped.shape) # (36, 28, 28, 1)
    xs_reshaped = ( xs_reshaped - 0.5 ) * 2.0
    xs_reshaped = np.lib.pad(xs_reshaped, ( (0,0), (2,2), (2,2), (0,0) ) , 'constant', constant_values = (-1, -1))
#     print(xs_reshaped.shape) # (36, 32, 32, 1)
    _, dLoss = sess.run([update_D, d_loss], feed_dict = { z_in:zs, real_in: xs_reshaped })
    _, gLoss = sess.run([update_G, g_loss], feed_dict = { z_in:zs  })
    _, gLoss = sess.run([update_G, g_loss], feed_dict = { z_in:zs  })
    
    if i % 2 == 0:
        print("Gem Loss: " + str(gLoss) + " Disc Loss: " + str(dLoss))
        z2 = np.random.uniform(-1.0, 1.0, size = [batch_size, z_size]).astype(np.float32)
        newZ = sess.run(Gz, feed_dict = { z_in: z2} )
        print(newZ[0].shape)
        
        for j in range(batch_size):
            count = count + 1
            image = newZ[j]
            # path where you want to store the images individually generated by the Generator during the TRAINING PHASE.
            # These are expected to make little or no sense at all semantically.
            if not os.path.exists('/home/paragjain/Desktop/GAN_Trial/generator_output_image'):
                os.makedirs('/home/paragjain/Desktop/GAN_Trial/generator_output_image')
            cv2.imwrite('/home/paragjain/Desktop/GAN_Trial/generator_output_image/output'+str(count)+'.jpeg', image)
        
#             if count < 10:
#                 cv2.imshow('count'+str(count), image)
#                 cv2.waitKey(0)
#                 cv2.destroyAllWindows()
# #             cv2.imshow('image', image)
#             cv2.waitKey(0)
#             cv2.destroyAllWindows()
#         print(newZ.shape) # (36, 32, 32, 1)
        newZ_reshaped = np.reshape(newZ[0:36], [36,32,32])
#         real_or_fake = [newZ[0]]
#         real_or_fake_probablity = sess.run(Dx, feed_dict={real_in:real_or_fake})
#         print("real_or_fake_probablity" , real_or_fake_probablity)

          # path where you want to store the batch of images generated by the Generator during the TRAINING PHASE.
         # These are expected to make little or no sense at all semantically.
           
        save_images(newZ_reshaped, [6,6], '/home/paragjain/Desktop/GAN_Trial/sample_directory'+'/fig'+ str(i)+".png")
#         cv2.imwrite('/home/paragjain/Desktop/GAN Trial/sample_directory')

    if i % 9 == 0 and i!=0:
        # path where you want to store the model so that you can retrieve it later.
        if not os.path.exists('/home/paragjain/Desktop/GAN_Trial/model_directory'):
            os.makedirs('/home/paragjain/Desktop/GAN_Trial/model_directory')
        save_path =  '/home/paragjain/Desktop/GAN_Trial/model_directory' + '/model-' + str(i) + '.cptk'
        print(save_path)
        saver.save(sess, save_path)
        print("Saved Model")

Gem Loss: 0.496868 Disc Loss: 1.40135
(32, 32, 1)
Gem Loss: 0.259155 Disc Loss: 1.78214
(32, 32, 1)
Gem Loss: 0.210909 Disc Loss: 2.11912
(32, 32, 1)
Gem Loss: 0.19958 Disc Loss: 2.19322
(32, 32, 1)
Gem Loss: 0.221899 Disc Loss: 2.20146
(32, 32, 1)
/home/paragjain/Desktop/GAN_Trial/model_directory/model-9.cptk
Saved Model
Gem Loss: 0.263144 Disc Loss: 2.09193
(32, 32, 1)


In [23]:
sess = tf.Session()
saver = tf.train.Saver()

# Retrieving the saved model
ckpt = tf.train.get_checkpoint_state('/home/paragjain/Desktop/GAN_Trial/model_directory')
saver.restore(sess, ckpt.model_checkpoint_path)

zs = np.random.uniform(-1.0, 1.0, size = [batch_size, z_size]).astype(np.float32)
newZ = sess.run(Gz, feed_dict = { z_in: zs })
                
if not os.path.exists('/home/paragjain/Desktop/GAN_Trial/model_learnt_images'):
             os.makedirs('/home/paragjain/Desktop/GAN_Trial/model_learnt_images')

                
newZ_reshaped = np.reshape(newZ[0:36], [36,32,32])

# Path where you would want to store the Generated Image
if not os.path.exists('/home/paragjain/Desktop/GAN_Trial/sample_learnt_images'):
    os.makedirs('/home/paragjain/Desktop/GAN_Trial/sample_learnt_images')
                
save_images(newZ_reshaped, [6,6], '/home/paragjain/Desktop/GAN_Trial/sample_learnt_images'+'/final.jpg')

INFO:tensorflow:Restoring parameters from /home/paragjain/Desktop/GAN Trial/model_directory/model-9.cptk
